In [2]:
import pandas as pd

In [3]:
# https://files.grouplens.org/datasets/movielens/ml-25m.zip

movies = pd.read_csv(r"C:\Users\User\Downloads\movies.csv")

In [4]:
movies

,movieId,title,genres;;
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy;;
1,2,Jumanji (1995),Adventure|Children|Fantasy;;
2,3,Grumpier Old Men (1995),Comedy|Romance;;
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance;;
4,5,Father of the Bride Part II (1995),Comedy;;
...,...,...,...
62418,209157,We (2018),Drama;;
62419,209159,Window of the Soul (2001),Documentary;;
62420,209163,Bad Poems (2018),Comedy|Drama;;
62421,209169,A Girl Thing (2001),(no genres listed);;


In [5]:
import re

def clean_title(title):
    if isinstance(title, str):
        return re.sub("[^a-zA-Z0-9\s]", "", title)
    else:
        return ""

movies["clean_title"] = movies["title"].fillna("").apply(clean_title)


In [6]:
movies

,movieId,title,genres;;,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy;;,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy;;,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance;;,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance;;,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy;;,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama;;,We 2018
62419,209159,Window of the Soul (2001),Documentary;;,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama;;,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed);;,A Girl Thing 2001


In [7]:
!pip install --upgrade --force-reinstall scikit-learn

  Using cached scikit_learn-1.4.1.post1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached scipy-1.12.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.4.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.4.1.post1-cp312-cp312-win_amd64.whl (10.6 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
Using cached scipy-1.12.0-cp312-cp312-win_amd64.whl (45.8 MB)
Using cached threadpoolctl-3.4.0-py3-none-any.whl (17 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.4.0
    Uninstalling threadpoolctl-3.4.0:
      Successfully uninstalled threadpoolctl-3.4.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  At

In [8]:
import sklearn

In [9]:
import sys
print(sys.executable)

c:\users\user\appdata\local\programs\python\python39\python.exe


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = "Toy Story 1995"
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices][::-1]
    return results

In [14]:
query_vec

<1x149922 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [15]:
similarity

array([1.        , 0.10017521, 0.0675077 , ..., 0.        , 0.        ,
       0.        ])

In [16]:
indices

array([20497, 59767,  3021, 14813,     0], dtype=int64)

In [19]:
results

,movieId,title,genres;;,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy;;,Toy Story 1995
14813,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IM...,Toy Story 3 2010
3021,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy;;,Toy Story 2 1999
59767,201588,Toy Story 4 (2019),Adventure|Animation|Children|Comedy;;,Toy Story 4 2019
20497,106022,Toy Story of Terror (2013),Animation|Children|Comedy;;,Toy Story of Terror 2013


In [20]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value = "Toy Story",
    description = "Movie Title:",
    disabled = False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output() 
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')

display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [21]:
ratings = pd.read_csv(r"C:\Users\User\Downloads\ratings.csv")

In [22]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
1048570,7045,4447,3.5,1164258032
1048571,7045,4720,4.0,1164257756
1048572,7045,4857,4.0,1164242753
1048573,7045,4886,5.0,1168033506


In [23]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [24]:
movie_id = 1

In [25]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [26]:
similar_users

array([  36,   75,   86,   90,   93,   95,   96,   98,  111,  120,  127,
        143,  152,  158,  160,  162,  171,  186,  188,  211,  217,  229,
        230,  235,  249,  257,  259,  297,  298,  302,  323,  329,  355,
        359,  369,  371,  381,  392,  402,  411,  428,  435,  439,  447,
        449,  468,  469,  477,  484,  513,  519,  537,  540,  541,  548,
        551,  553,  561,  567,  573,  582,  593,  607,  609,  611,  623,
        624,  626,  628,  631,  644,  653,  654,  670,  683,  686,  694,
        697,  702,  709,  727,  733,  741,  749,  752,  765,  768,  773,
        785,  791,  793,  796,  803,  805,  807,  811,  830,  834,  839,
        848,  856,  896,  904,  905,  911,  927,  947,  950,  956,  966,
        969,  986,  997, 1007, 1010, 1013, 1036, 1038, 1042, 1065, 1079,
       1092, 1096, 1101, 1118, 1123, 1131, 1138, 1140, 1141, 1143, 1146,
       1150, 1159, 1166, 1167, 1169, 1171, 1176, 1179, 1192, 1196, 1198,
       1199, 1200, 1228, 1230, 1232, 1240, 1242, 12

In [27]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [28]:
similar_user_recs

5101          1
5105         34
5111        110
5114        150
5127        260
           ... 
1048561    3996
1048563    4022
1048564    4027
1048565    4223
1048573    4886
Name: movieId, Length: 54307, dtype: int64

In [29]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .1]

In [30]:
similar_user_recs

movieId
1        1.000000
318      0.430189
260      0.374843
356      0.339623
296      0.338365
           ...   
1278     0.101887
780      0.101887
33794    0.101887
953      0.100629
1246     0.100629
Name: count, Length: 101, dtype: float64

In [31]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [32]:
all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [33]:
all_users_recs

movieId
318      0.348655
296      0.283321
2571     0.241814
356      0.234743
593      0.225211
           ...   
953      0.042583
1278     0.039508
50872    0.038432
78499    0.036434
2355     0.024750
Name: count, Length: 101, dtype: float64

In [34]:
rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis = 1)
rec_percentages.columns = ["similar", "all"]

In [35]:
rec_percentages

,similar,all
movieId,,
1,1.000000,0.122214
318,0.430189,0.348655
260,0.374843,0.211376
356,0.339623,0.234743
296,0.338365,0.283321
...,...,...
1278,0.101887,0.039508
780,0.101887,0.052882
33794,0.101887,0.064566


In [36]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [37]:
rec_percentages = rec_percentages.sort_values("score", ascending = False)

In [38]:
rec_percentages

,similar,all,score
movieId,,,
1,1.000000,0.122214,8.182390
3114,0.267925,0.051038,5.249545
2355,0.113208,0.024750,4.574007
78499,0.153459,0.036434,4.212032
588,0.221384,0.067025,3.302983
...,...,...,...
4973,0.125786,0.106226,1.184137
2329,0.110692,0.094696,1.168913
2858,0.192453,0.165257,1.164563


In [40]:
# Assuming you want to merge on the index
result = pd.concat([rec_percentages.head(10), movies], axis=1)

In [48]:
result.head(10)

,similar,all,score,movieId,title,genres;;,clean_title
1,1.000000,0.122214,8.182390,2,Jumanji (1995),Adventure|Children|Fantasy;;,Jumanji 1995
3114,0.267925,0.051038,5.249545,"3207,""Snows of Kilimanjaro, The (1952)"",Advent...",NaN,NaN,
2355,0.113208,0.024750,4.574007,2446,In Dreams (1999),Horror|Thriller;;,In Dreams 1999
78499,0.153459,0.036434,4.212032,NaN,NaN,NaN,NaN
588,0.221384,0.067025,3.302983,596,Pinocchio (1940),Animation|Children|Fantasy|Musical;;,Pinocchio 1940
4886,0.212579,0.067487,3.149941,4992,Kate & Leopold (2001),Comedy|Romance;;,Kate Leopold 2001
595,0.188679,0.064258,2.936264,"603,""Bye Bye, Love (1995)"",Comedy;;",NaN,NaN,
6377,0.191195,0.067333,2.839551,"6500,""Satanic Rites of Dracula, The (1974)"",Ho...",NaN,NaN,
1073,0.140881,0.050423,2.793987,1100,Days of Thunder (1990),Action|Drama|Romance;;,Days of Thunder 1990
364,0.242767,0.088701,2.736917,369,Mrs. Parker and the Vicious Circle (1994),Drama;;,Mrs Parker and the Vicious Circle 1994


In [59]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis = 1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    
    rec_percentages = rec_percentages.sort_values("score", ascending = False)
    return pd.concat([rec_percentages.head(10), movies], axis=1) [["score", "title", "genres;;"]]

In [60]:
movie_name_input = widgets.Text(
    value="Toy Story",
    description="Movie Title:",
    disabled = False
)

recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))
            
movie_name_input.observe(on_type, names="value")

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()